In [5]:
import matplotlib.pyplot as plt

import drjit as dr
import mitsuba as mi
import os
import imageio.v2 as imageio

#mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')
mi.set_variant('llvm_ad_rgb')
from mitsuba import ScalarTransform4f as T

sensor_count = 3
sensors = []
Resolution = 128

# for i in range(sensor_count):
angle = 180.0 / sensor_count - 90.0
sensor_rotation = T().rotate([0, 1, 0], angle)
sensor_to_world = T().look_at(target=[0, 0, 0], origin=[0, -2, 2], up=[0, 1, 0])
sensors.append(mi.load_dict({
        'type': 'perspective',
        'fov': 45,
        'to_world': sensor_rotation @ sensor_to_world,
        'film': {
            'type': 'hdrfilm',
            'width': Resolution, 'height': Resolution,
            'filter': {'type': 'gaussian'}
        }
    }))

output_dir = "frames" 
os.makedirs(output_dir, exist_ok=True)
image_paths = []

for frame_idx in range(100):
    scene_dict = {
    'type': 'scene',
    'integrator': {'type': 'prbvolpath'},
    'object': {
        'type': 'cube',
        'bsdf': {'type': 'null'},
        'interior': {
            'type': 'heterogeneous',
            'sigma_t': {
                'type': 'gridvolume',
                'filename': f'../frame_{frame_idx:06d}.vol',
                'to_world': T().rotate([1, 0, 0], 0).scale(2).translate(-0.5)
            },
            'scale': 40
        }
    },
    'emitter': {'type': 'constant'}
    }

    scene_ref = mi.load_dict(scene_dict)

    # Number of samples per pixel for reference images
    ref_spp = 512
    ref_images = mi.render(scene_ref, sensor=sensors[0], spp=ref_spp)
    #fig, axs = plt.subplots(1, 1, figsize=(42, 12))
    bitmap = mi.util.convert_to_bitmap(ref_images)
    out_path = os.path.join(output_dir, f"frame_{frame_idx:06d}.png")
    bitmap.write(out_path)
    image_paths.append(out_path)

video_path = "output_video.mp4"
with imageio.get_writer(video_path, fps=24) as writer:
    for path in image_paths:
        writer.append_data(imageio.imread(path))

# scene_dict = {
#     'type': 'scene',
#     'integrator': {'type': 'prbvolpath'},
#     'object': {
#         'type': 'cube',
#         'bsdf': {'type': 'null'},
#         'interior': {
#             'type': 'heterogeneous',
#             'sigma_t': {
#                 'type': 'gridvolume',
#                 'filename': '../frame_000099.vol',
#                 'to_world': T().rotate([1, 0, 0], 0).scale(2).translate(-0.5)
#             },
#             'scale': 40
#         }
#     },
#     'emitter': {'type': 'constant'}
# }

# scene_ref = mi.load_dict(scene_dict)

# # Number of samples per pixel for reference images
# ref_spp = 512
# ref_images = mi.render(scene_ref, sensor=sensors[0], spp=ref_spp)
# fig, axs = plt.subplots(1, 1, figsize=(42, 12))
# # for i in range(sensor_count):
# #     axs[i].imshow(mi.util.convert_to_bitmap(ref_images[i]))
# #     axs[i].axis('off')

# axs.imshow(mi.util.convert_to_bitmap(ref_images))
# axs.axis('off')